In [1]:
import os
os.chdir("../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ee
ee.Initialize()
import geemap
print(ee.__version__)

0.1.360


In [2]:
dc = ee.FeatureCollection('TIGER/2018/States').filterMetadata('NAME', 'equals', 'District of Columbia').geometry()

## LANDSAT

In [3]:
Map = geemap.Map(center=(38.897957, -77.036560), zoom=8)
landsat_imgs = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
           .filterDate('2010-01-01', '2021-12-31')
           .filterBounds(dc)
           .filter('CLOUD_COVER < 5'))

vis_params = {'min': 0, 'max': 4000, 'bands': ['B5', 'B4', 'B3']}

Map.addLayer(landsat_imgs, vis_params, "Landsat-8")
Map

Map(center=[38.897957, -77.03656], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [4]:
def ndvi_func(i):
    ndvi = i.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_imgs.map(ndvi_func)

In [5]:
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange(y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), dc, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
display(df)

,NDVI,year
0,0.255553,2013
1,0.415431,2014
2,0.425041,2015
3,0.329087,2016
4,0.348490,2017
5,0.326505,2018
6,0.429704,2019
7,0.492855,2020
8,0.388530,2021
